## Programming Exercise 3: Neural Networks Learning

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io #Used to load the OCTAVE *.mat files
import scipy.misc #Used to show matrix as an image
import matplotlib.cm as cm #Used to display images in a specific colormap
import random #To pick random images to display
from scipy.special import expit #Vectorized sigmoid function

### 1 Neural Networks

#### 1.1 Visualizing the data

In [ ]:
#Note this is actually a symlink... same data as last exercise,
#so there's no reason to add another 7MB to my github repo...
datafile = 'data/ex4data1.mat'
mat = scipy.io.loadmat( datafile )
X, y = mat['X'], mat['y']
#Insert a column of 1's to X as usual
X = np.insert(X,0,1,axis=1)
print "'y' shape: %s. Unique elements in y: %s"%(mat['y'].shape,np.unique(mat['y']))
print "'X' shape: %s. X[0] shape: %s"%(X.shape,X[0].shape)
#X is 5000 images. Each image is a row. Each image has 400 pixels unrolled (20x20)
#y is a classification for each image. 1-10, where "10" is the handwritten "0"

In [ ]:
def getDatumImg(row):
    """
    Function that is handed a single np array with shape 1x400,
    crates an image object from it, and returns it
    """
    width, height = 20, 20
    square = row[1:].reshape(width,height)
    return square.T
    
def displayData(indices_to_display = None):
    """
    Function that picks 100 random rows from X, creates a 20x20 image from each,
    then stitches them together into a 10x10 grid of images, and shows it.
    """
    width, height = 20, 20
    nrows, ncols = 10, 10
    if not indices_to_display:
        indices_to_display = random.sample(range(X.shape[0]), nrows*ncols)
        
    big_picture = np.zeros((height*nrows,width*ncols))
    
    irow, icol = 0, 0
    for idx in indices_to_display:
        if icol == ncols:
            irow += 1
            icol  = 0
        iimg = getDatumImg(X[idx])
        big_picture[irow*height:irow*height+iimg.shape[0],icol*width:icol*width+iimg.shape[1]] = iimg
        icol += 1
    fig = plt.figure(figsize=(6,6))
    img = scipy.misc.toimage( big_picture )
    plt.imshow(img,cmap = cm.Greys_r)

In [ ]:
displayData()

#### 1.2 Model representation

In [ ]:
#You have been provided with a set of network parameters (Θ(1),Θ(2)) 
#already trained by us. These are stored in ex4weights.mat
datafile = 'data/ex4weights.mat'
mat = scipy.io.loadmat( datafile )
Theta1, Theta2 = mat['Theta1'], mat['Theta2']
# The matrices Theta1 and Theta2 will now be in your workspace
# Theta1 has size 25 x 401
# Theta2 has size 10 x 26
print "Theta1 has shape:",Theta1.shape
print "Theta2 has shape:",Theta2.shape

In [ ]:
# These are some global variables I'm suing to ensure the sizes
# of various matrices are correct
#these are NOT including bias nits
input_layer_size = 400
hidden_layer_size = 25
output_layer_size = 10 

In [ ]:
#blah = np.ones((3,4))
#np.insert(a, 1, 5, axis=1)
#print np.insert(blah,0,1,axis=1)

In [ ]:
import itertools

def flattenParams(thetas_list):
    """
    Hand this function a list of theta matrices, and it will flatten it
    into one long (n,1) shaped numpy array
    """
    flattened_list = [ mytheta.flatten() for mytheta in thetas_list ]
    combined = list(itertools.chain.from_iterable(flattened_list))
    assert len(combined) == (input_layer_size+1)*hidden_layer_size + \
                            (hidden_layer_size+1)*output_layer_size
    return np.array(combined).reshape((len(combined),1))

def reshapeParams(flattened_array):
    theta1 = flattened_array[:(input_layer_size+1)*hidden_layer_size] \
            .reshape((hidden_layer_size,input_layer_size+1))
    theta2 = flattened_array[(input_layer_size+1)*hidden_layer_size:] \
            .reshape((output_layer_size,hidden_layer_size+1))
    
    return [ theta1, theta2 ]

myThetas = [ Theta1, Theta2 ]
print Theta1.shape
print Theta2.shape
flattened = flattenParams(myThetas)
print "flattend shape = ",flattened.shape
reshaped = reshapeParams(flattened)
print "reshaped length = ",len(reshaped)
print "reshaped inner shapes: ",[inner.shape for inner in reshaped]

#### 1.3 Feedforward and cost function

In [ ]:
def computeCost(mythetas_flattened,myX,myy,mylambda=0.):
    """
    This function takes in:
        1) a flattened vector of theta parameters (each theta would go from one
           NN layer to the next), the thetas do not include the bias unit.
           The thetas are NOT "unrolled" in this function
        2) the training set matrix X, which contains the bias unit first column
        3) the label vector y, which has one column
    It loops over training points (recommended by the professor, as the linear
    algebra version is "quite complicated") and, for each, it:
        1) loops over the each training point, and for each, it
            2) constructs a new "y" vector, with 10 rows and 1 column, 
                with one non-zero entry corresponding to that iteration
            3) computes the cost given that y- vector and that training point
        4) sums up all of the costs
        5) computes a regularization term
    """
    
    # First unroll the parameters
    mythetas = reshapeParams(mythetas_flattened)
    
    #This is what will accumulate the total cost
    total_cost = 0.
    
    m = myX.shape[0] #5000

    # Loop over the training points (rows in myX, already contain bias unit)
    for irow in xrange(m):
        myrow = myX[irow]
                
        # First compute the hypothesis (this is a (10,1) vector
        # of the hypothesis for each possible y-value)
        # propagateForward returns (zs, activations) for each layer
        # so propagateforward[-1][1] means "activation for -1st (last) layer"
        myhs = propagateForward(myrow,mythetas)[-1][1]

        # Construct a 10x1 "y" vector with all zeros and only one "1" entry
        # note here if the hand-written digit is "0", then that corresponds
        # to a y- vector with 1 in the 10th spot (different from what the
        # homework suggests)
        tmpy  = np.zeros((10,1))
        tmpy[myy[irow]-1] = 1
        
        # Compute the cost for this point and y-vector
        mycost = -tmpy.T.dot(np.log(myhs))-(1-tmpy.T).dot(np.log(1-myhs))
     
        # Accumulate the total cost
        total_cost += mycost
  
    # Normalize the total_cost, cast as float
    total_cost = float(total_cost) / m
    
    # Compute the regularization term
    total_reg = 0.
    for mytheta in mythetas:
        total_reg += np.sum(mytheta*mytheta) #element-wise multiplication
    total_reg *= float(mylambda)/(2*m)
        
    return total_cost + total_reg
       

def propagateForward(row,Thetas):
    """
    Function that given a list of Thetas, propagates the
    Row of features forwards, assuming the features ALREADY
    include the bias unit in the input layer, and the 
    Thetas need the bias unit added to features between each layer

    The output is a vector with element [0] for the hidden layer,
    and element [1] for the output layer
        -- Each element is a tuple of (zs, as)
        -- where "zs" and "as" have shape (# of units in that layer, 1)
    
    ***The 'activations' are the same as "h", but this works for many layers
    (hence a vector of thetas, not just one theta)
    Also, "h" is vectorized to do all rows at once...
    this function takes in one row at a time***
    """
    
    #HEY DAVID REWRITE THIS SHITTY HANDLE ON WHETHER THETAS WAS FLATTENED OR NOT
    #Theta1 has shape: (25, 401)
    #Theta2 has shape: (10, 26)
    #Thetas = np.array(Thetas)
    #shapedTheta0 = np.zeros((25,400))
    #shapedTheta1 = np.zeros((10,25))
    #if Thetas.shape[0] == 10285:
    #    np.insert(shapedTheta0,0,0,axis=1)
    #    np.insert(shapedTheta1,0,0,axis=1)
    #    shapedTheta0 = Thetas[:10025].reshape((25,401))
    #    shapedTheta1 = Thetas[10025:].reshape((10,26))
     
        
    #if Thetas.shape[0] == 10250:
    #    shapedTheta0 = Thetas[:10000].reshape((25,400))
    #    shapedTheta1 = Thetas[10000:].reshape((10,25))
    #    #add bias units
    #    np.insert(shapedTheta0,0,1,axis=1)
    #    np.insert(shapedTheta1,0,1,axis=1)
    #    Thetas = []
    #    Thetas.append(shapedTheta0)
    #    Thetas.append(shapedTheta1)
   
    features = row
    zs_as_per_layer = []
    for i in xrange(len(Thetas)):      
        Theta = Thetas[i]
        z = Theta.dot(features).reshape((Theta.shape[0],1))
        a = expit(z)
        zs_as_per_layer.append( (z, a) )
        if i == len(Thetas)-1:
            return np.array(zs_as_per_layer)
        a = np.insert(a,0,1) #Add the bias unit
        features = a

In [ ]:
myThetas = [ Theta1, Theta2 ]
test = propagateForward(X[0],myThetas)
print test[0]
print test[1][1].shape
#for zs, activations in test:
#    print "zs shape = ",zs.shape
#    print "as shape = ",activations.shape

In [ ]:
#Once you are done, using the loaded set of parameters Theta1 and Theta2,
#you should see that the cost is about 0.287629
myThetas = [ Theta1, Theta2 ]

#Note I flatten the thetas vector before handing it to the computeCost routine,
#as per the input format of the computeCost function.
#It does the unrolling/reshaping itself
print computeCost(flattenParams(myThetas),X,y)

#### 1.4 Regularized cost function

In [ ]:
#Once you are done, using the loaded set of parameters Theta1 and Theta2,
#and lambda = 1, you should see that the cost is about 0.383770
myThetas = [ Theta1, Theta2 ]
print computeCost(flattenParams(myThetas),X,y,mylambda=1.)

### 2 Backpropagation

#### 2.1 Sigmoid gradient

In [ ]:
def sigmoidGradient(z):
    dummy = expit(z)
    return dummy*(1-dummy)

#### 2.2 Random initialization

In [ ]:
def genRandThetas():
    epsilon_init = 0.12
    theta1_shape = (hidden_layer_size, input_layer_size+1)
    theta2_shape = (output_layer_size, hidden_layer_size+1)
    rand_thetas = [ np.random.rand( *theta1_shape ) * 2 * epsilon_init - epsilon_init, \
                    np.random.rand( *theta2_shape ) * 2 * epsilon_init - epsilon_init]
    return rand_thetas

In [ ]:
#myThetas = [ Theta1, Theta2 ]
#myhs = propagateForward(X[0],myThetas)
#print myhs[0].shape

#### 2.3 Backpropagation


In [ ]:
def backPropagate(myX,myy,mythetas):
    #input_layer_size = 400
    #hidden_layer_size = 25
    #output_layer_size = 10 
    m = myX.shape[0] #5000
    
    
    
    ### Question: what shape should Delta have? Does it include hidden layers?
    
    
    
    
    Delta1 = np.zeros((hidden_layer_size,input_layer_size))
    Delta2 = np.zeros((output_layer_size,hidden_layer_size))
    m = 1
    # Loop over the training points (rows in myX, already contain bias unit)
    for irow in xrange(m):
        myrow = myX[irow]
        a1 = myrow[1:].reshape((400,1))
        z1 = expit(a1)#[1:].reshape((400,1))
        #print "z1 shape is ",z1.shape
        # propagateForward returns (zs, activations) for each layer excluding the input layer
        temp = propagateForward(myrow,mythetas)
        z2 = temp[0][0]
        #print "z2 shape is ",z2.shape
        a2 = temp[0][1]
        z3 = temp[1][0]
        #print "z3 shape is ",z3.shape
        a3 = temp[1][1]
        tmpy  = np.zeros((10,1))
        tmpy[myy[irow]-1] = 1
        delta3 = a3 - tmpy 
        delta2 = mythetas[1].T.dot(delta3)[1:]*sigmoidGradient(z2) #remove 0th element
        delta1 = mythetas[0].T.dot(delta2)[1:]*sigmoidGradient(z1) #remove 0th element
        Delta1 += delta2.dot(a1.T)
        Delta2 += delta3.dot(a2.T)
        print "delta1 shape is ",delta1.shape
        print "delta2 shape is ",delta2.shape
        print "Delta2 shape is ",delta3.dot(a2.T).shape
        print "Delta1 shape is ",delta2.dot(a1.T).shape
    D1 = Delta1/float(m)
    D2 = Delta2/float(m)
    return D1, D2

In [ ]:
D1, D2 = backPropagate(X,y,myThetas)
print D1.shape
print D2.shape

#### 2.3 Gradient checking

In [ ]:
def checkGradient(mythetas,myX,myy,mylambda=0.):
    myeps = 0.0001
    myshapes = [ myTheta[:,1:].shape for myTheta in mythetas ]
    print myshapes
    flattened = list(mythetas[0][:,1:].flatten()) + list(mythetas[1][:,1:].flatten())
    flattened = np.array(flattened).reshape((len(flattened),1))
    print flattened.shape
    n_elems = len(flattened)
    print n_elems
    grad = np.zeros((n_elems,1))
    for x in xrange(1):#n_elems):
        epsvec = np.zeros((n_elems,1))
        epsvec[x] = myeps
        #print "flattened shape is ",flattened.shape
        cost_high = computeCost(flattened + epsvec,myX,myy,mylambda)
        cost_low  = computeCost(flattened - epsvec,myX,myy,mylambda)
        mygrad = (cost_high - cost_low) / float(2*myeps)
        print mygrad
        grad[x] = mygrad
    return grad

In [ ]:
#This is all c/p old stuff from the previous week(s) and should be removed

In [ ]:

def computeCostDELETEME(mytheta,myX,myy,mylambda = 0.):
    m = myX.shape[0] #5000
    myh = h(mytheta,myX) #shape: (5000,1)
    term1 = np.log( myh ).dot( -myy.T ) #shape: (5000,5000)
    term2 = np.log( 1.0 - myh ).dot( 1 - myy.T ) #shape: (5000,5000)
    left_hand = (term1 - term2) / m #shape: (5000,5000)
    right_hand = mytheta.T.dot( mytheta ) * mylambda / (2*m) #shape: (1,1)
    return left_hand + right_hand #shape: (5000,5000)

def computeCost(mythetas,myX,myy)
    """
    This function takes in:
        1) a vector of thetas (each theta goes from one
           NN layer to the next), the thetas do not include the bias unit.
        2) the training set matrix X, which contains the bias unit first column
        3) the label vector y, which has one column
    It loops over training points (recommended by the professor, as the linear
    algebra version is "quite complicated") and, for each, it:
        1) loops over the each training point, and for each, it
            2) constructs a new "y" vector, with 10 rows and 1 column, 
                with one non-zero entry corresponding to that iteration
            3) computes the cost given that y- vector and that training point
        4) sums up all of the costs
        
            2b) computes the cost for each of 10 possible outcomes
    """
    m = myX.shape[0] #5000
    for training_pt in myX

def propagateForward(row,Thetas):
    """
    Function that given a list of Thetas, propagates the
    Row of features forwards, assuming the features already
    include the bias unit in the input layer, and the 
    Thetas need the bias unit added to features between each layer
    
    ***This is the same as "h", but this works for many layers
    (hence a vector of thetas, not just one theta)
    Also, "h" is vectorized to do all rows at once...
    this function takes in one row at a time***
    
    """
    features = row
    for i in xrange(len(Thetas)):
        Theta = Thetas[i]
        z = Theta.dot(features)
        a = expit(z)
        if i == len(Thetas)-1:
            return a
        a = np.insert(a,0,1) #Add the bias unit
        features = a

def predictNN(row,Thetas):
    """
    Function that takes a row of features, propagates them through the
    NN, and returns the predicted integer that was hand written
    """
    classes = range(1,10) + [10]
    output = propagateForward(row,Thetas)
    return classes[np.argmax(np.array(output))]

In [ ]:
myThetas = [ Theta1, Theta2 ]
#Loop over all of the rows in X (all of the handwritten images)
#and predict what digit is written. Check if it's correct, and
#compute an efficiency.
for irow in xrange(X.shape[0]):
    if predictNN(X[irow],myThetas) == int(y[irow]): 
        n_correct += 1
    else: incorrect_indices.append(irow)
print "Training set accuracy: %0.1f%%"%(100*(n_correct/n_total))

In [ ]:
# Once you are done, ex4.m will call your nnCostFunction 
# using the loaded set of parameters for Theta1 and Theta2. 
# You should see that the cost is about 0.287629.

#### 1.4 Regularized cost function

In [ ]:
#Hypothesis function and cost function for logistic regression
def h(mytheta,myX): #Logistic hypothesis function
    return expit(np.dot(myX,mytheta))


In [ ]:
#An alternative to OCTAVE's 'fmincg' we'll use some scipy.optimize function, "fmin_cg"
#This is more efficient with large number of parameters.
#In the previous homework, I didn't have to compute the cost gradient because
#the scipy.optimize function did it for me with some kind of interpolation...
#However, fmin_cg needs the gradient handed do it, so I'll implement that here
def costGradient(mytheta,myX,myy,mylambda = 0.):
    m = myX.shape[0]
    #Tranpose y here because it makes the units work out in dot products later
    #(with the way I've written them, anyway)
    beta = h(mytheta,myX)-myy.T #shape: (5000,5000)

    #regularization skips the first element in theta
    regterm = mytheta[1:]*(mylambda/m) #shape: (400,1)

    grad = (1./m)*np.dot(myX.T,beta) #shape: (401, 5000)
    #regularization skips the first element in theta
    grad[1:] = grad[1:] + regterm
    return grad #shape: (401, 5000)

In [ ]:
from scipy import optimize

def optimizeTheta(mytheta,myX,myy,mylambda=0.):
    result = optimize.fmin_cg(computeCost, fprime=costGradient, x0=mytheta, \
                              args=(myX, myy, mylambda), maxiter=50, disp=False,\
                              full_output=True)
    return result[0], result[1]

In [ ]:
#Note: I spent a LONG time trying to optimize everything. Initially training 10 classes
#took about 5 minutes. Now I've got it down to taking ~5 seconds total!
def buildTheta():
    """
    Function that determines an optimized theta for each class
    and returns a Theta function where each row corresponds
    to the learned logistic regression params for one class
    """
    mylambda = 0.
    initial_theta = np.zeros((X.shape[1],1)).reshape(-1)
    Theta = np.zeros((10,X.shape[1]))
    for i in xrange(10):
        iclass = i if i else 10 #class "10" corresponds to handwritten zero
        print "Optimizing for handwritten number %d..."%i
        logic_Y = np.array([1 if x == iclass else 0 for x in y])#.reshape((X.shape[0],1))
        itheta, imincost = optimizeTheta(initial_theta,X,logic_Y,mylambda)
        Theta[i,:] = itheta
    print "Done!"
    return Theta

In [ ]:
Theta = buildTheta()

In [ ]:
def predictOneVsAll(myTheta,myrow):
    """
    Function that computes a hypothesis for an individual image (row in X)
    and returns the predicted integer corresponding to the handwritten image
    """
    classes = [10] + range(1,10)
    hypots  = [0]*len(classes)
    #Compute a hypothesis for each possible outcome
    #Choose the maximum hypothesis to find result
    for i in xrange(len(classes)):
        hypots[i] = h(myTheta[i],myrow)
    return classes[np.argmax(np.array(hypots))]    

In [ ]:
# "You should see that the training set accuracy is about 94.9%"
n_correct, n_total = 0., 0.
incorrect_indices = []
for irow in xrange(X.shape[0]):
    n_total += 1
    if predictOneVsAll(Theta,X[irow]) == y[irow]: 
        n_correct += 1
    else: incorrect_indices.append(irow)
print "Training set accuracy: %0.1f%%"%(100*(n_correct/n_total))

In [ ]:
#Let's have a look at the ones we get wrong:
displayData(incorrect_indices[:100])
displayData(incorrect_indices[100:200])
displayData(incorrect_indices[200:300])

### 2 Neural Networks

#### 2.1 Model representation

#### 2.2 Feedforward Propagation

In [ ]:
def propagateForward(row,Thetas):
    """
    Function that given a list of Thetas, propagates the
    Row of features forwards, assuming the features already
    include the bias unit in the input layer, and the 
    Thetas need the bias unit added to features between each layer
    """
    features = row
    for i in xrange(len(Thetas)):
        Theta = Thetas[i]
        z = Theta.dot(features)
        a = expit(z)
        if i == len(Thetas)-1:
            return a
        a = np.insert(a,0,1) #Add the bias unit
        features = a

def predictNN(row,Thetas):
    """
    Function that takes a row of features, propagates them through the
    NN, and returns the predicted integer that was hand written
    """
    classes = range(1,10) + [10]
    output = propagateForward(row,Thetas)
    return classes[np.argmax(np.array(output))]

In [ ]:
# "You should see that the accuracy is about 97.5%"
myThetas = [ Theta1, Theta2 ]
n_correct, n_total = 0., 0.
incorrect_indices = []
#Loop over all of the rows in X (all of the handwritten images)
#and predict what digit is written. Check if it's correct, and
#compute an efficiency.
for irow in xrange(X.shape[0]):
    n_total += 1
    if predictNN(X[irow],myThetas) == int(y[irow]): 
        n_correct += 1
    else: incorrect_indices.append(irow)
print "Training set accuracy: %0.1f%%"%(100*(n_correct/n_total))

In [ ]:
#Pick some of the images we got WRONG and look at them, just to see
for x in xrange(5):
    i = random.choice(incorrect_indices)
    fig = plt.figure(figsize=(3,3))
    img = scipy.misc.toimage( getDatumImg(X[i]) )
    plt.imshow(img,cmap = cm.Greys_r)
    predicted_val = predictNN(X[i],myThetas)
    predicted_val = 0 if predicted_val == 10 else predicted_val
    fig.suptitle('Predicted: %d'%predicted_val, fontsize=14, fontweight='bold')